<a href="https://colab.research.google.com/github/artiboi2002/dp-poisoning-attacks/blob/main/dp_poisioning_attacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from opacus import PrivacyEngine
import time

transform = transforms.ToTensor() #converting MNIST Image into tensor of 4 dimensions(image, height, wdith, color channel)
#Train Data
trainData = datasets.MNIST(root ='/cnn_data', train = True, download = True, transform = transform)
#Test Data
testData = datasets.MNIST(root = '/cnn_data', train = False, download = True, transform = transform)


In [19]:
class ConvolutionalNeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 6, 3, 1)
    self.conv2 = nn.Conv2d(6, 16, 3,1)
    # Fully connected layer
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x, 2,2) #2x2 kernal and a stride of 2
    x = F.relu(self.conv2(x)) #1 image, 6 output, 3 kernal, 1 stride
    x = F.max_pool2d(x, 2, 2)

    #Review to flatten in out
    x = x.view(-1, 16*5*5)

    #Fully connected layers
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)

    return F.log_softmax(x, dim = 1)

In [26]:
def trainingFunction(model, train_loader, epochs, criterion, optimizer):
  train_losses = []
  train_accuracies = []
  for i in range(epochs):
    total_loss = 0
    correct_pred = 0
    train_corr = 0 # Initialize train_corr here
    for b,(X_train, Y_train) in enumerate(train_loader):
      b += 1
      y_pred = model(X_train)
      loss = criterion(y_pred, Y_train)

      predicted = torch.max(y_pred.data, 1)[1]
      batch_corr = (predicted == Y_train).sum()
      train_corr += batch_corr

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if b % 600 == 0: # Changed from & to %
        print(f'epoch: {i} Batch {b} loss: {loss.item()}')
        train_losses.append(loss)

    train_accuracies.append(train_corr) # Append total train accuracy for the epoch
    train_corr = 0 # Reset train_corr for the next epoch

In [31]:
torch.manual_seed(41)
model = ConvolutionalNeuralNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.00001) #Smaller the learning rate the longer is gonna take to train
trainLoader = DataLoader(trainData, batch_size=10, shuffle = True)
testLoader = DataLoader(trainData, batch_size=10, shuffle = True)
epochs = 5

trainingFunction(model, trainLoader, epochs, criterion, optimizer)

UnboundLocalError: cannot access local variable 'train_corr' where it is not associated with a value